# 00. libraries

In [ ]:
!pip install torch
!pip install datasets
!pip install librosa
!pip install transformers
!pip install librosa soundfile
!pip install mutagen
!pip install librosa pydub
!pip install openai

In [ ]:
# import libraries
import datasets
from datasets import load_dataset, DatasetDict,  Audio
import pandas as pd
import os
import glob
import librosa
import io
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score
from transformers import WhisperModel, WhisperFeatureExtractor, AdamW
import torch
import torch.nn as nn
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.metrics import f1_score, classification_report, accuracy_score

In [ ]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Path to your file
file_path = '/content/drive/MyDrive/VUMC_postcleft/output_filename.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# Display the first few rows of the dataframe
# Replace ".mp3" with ".wav" in the "Filename" column
df['WAV_filename'] = df['File_Name'].str.replace('.mp3', '.wav')

# Create "WAV_folder" column by concatenating "_WAV" to the "folder" column
df['WAV_folder'] = df['folder'] + "_WAV"

df

# 01. audio to numeric value conversion(numpy array)

In [ ]:
import pandas as pd
import numpy as np
import librosa
import os

# Function to extract MFCCs from audio file (as provided earlier)
def extract_mfcc_features(file_path, n_mfcc=13):
    audio, sample_rate = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
    mfccs_scaled = np.mean(mfccs.T, axis=0)  # Taking the average across time
    return mfccs_scaled

# Load your DataFrame here with the appropriate method, e.g., pd.read_excel() if it's from an Excel file.
# df = pd.read_excel('/path_to_your_excel_file.xlsx')

# Add a new column to the DataFrame initialized with empty lists
df['numerical_representation'] = np.empty((len(df), 0)).tolist()

# Base paths for the control and cases WAV files
base_path = '/content/drive/MyDrive/VUMC_postcleft/WAV_PUBLIC_SAMPLES/'
controls_path = os.path.join(base_path, 'CONTROLS_WAV')
cases_path = os.path.join(base_path, 'CASES_WAV')

# Iterate over the rows of the DataFrame and calculate MFCCs
for index, row in df.iterrows():
    # Determine the folder based on the 'folder' column in the DataFrame
    folder_path = controls_path if row['folder'] == 'CONTROLS' else cases_path
    file_path = os.path.join(folder_path, row['WAV_filename'])

    # Check if the file exists before processing
    if os.path.exists(file_path):
        mfccs_scaled = extract_mfcc_features(file_path)
        df.at[index, 'numerical_representation'] = mfccs_scaled.tolist()
    else:
        print(f"File {file_path} does not exist.")

# Show the updated DataFrame
print(df[['File_Name', 'numerical_representation']].head())  # Show only relevant columns for brevity


                                           File_Name  \
0  ACPA 2 more in grammar school, and out little ...   
1  ACPA and I started out on the switch board and...   
2              ACPA playing in the snow is fun-3.mp3   
3                    Facebook (pick up the baby).mp3   
4  ACPA oh let me see, what can I tell you about ...   

                            numerical_representation  
0  [-370.3482666015625, 138.3352813720703, -31.24...  
1  [-364.57672119140625, 104.95849609375, -91.538...  
2  [-375.6810302734375, 66.8976821899414, -73.309...  
3  [-417.6043395996094, 58.079261779785156, -76.6...  
4  [-352.7203369140625, 140.18695068359375, -56.8...  


In [ ]:
df

,File_Name,Sampling_Rate_(Hz),Channels,Duration_(seconds),folder,hypernasality,original_text,OPENAI_Whisper_text,WAV_filename,WAV_folder,numerical_representation
0,"ACPA 2 more in grammar school, and out little ...",44100,1,3.27,CASES,1,"2 more in grammar school, and out little 3 yea...",We're in grammar school and I'm a three-year-o...,"ACPA 2 more in grammar school, and out little ...",CASES_WAV,"[-370.3482666015625, 138.3352813720703, -31.24..."
1,ACPA and I started out on the switch board and...,44100,1,3.89,CASES,1,and I started out on the switch board and two ...,And I started out on a switchboard answering p...,ACPA and I started out on the switch board and...,CASES_WAV,"[-364.57672119140625, 104.95849609375, -91.538..."
2,ACPA playing in the snow is fun-3.mp3,48000,1,2.18,CASES,1,playing in the snow is fun,Playing in the snow is fun.,ACPA playing in the snow is fun-3.wav,CASES_WAV,"[-375.6810302734375, 66.8976821899414, -73.309..."
3,Facebook (pick up the baby).mp3,44100,1,2.51,CASES,1,pick up the baby,Set up the maze!,Facebook (pick up the baby).wav,CASES_WAV,"[-417.6043395996094, 58.079261779785156, -76.6..."
4,"ACPA oh let me see, what can I tell you about ...",44100,1,4.13,CASES,1,"oh let me see, what can I tell you about them now","Oh, well, let's see, what can I tell you about...","ACPA oh let me see, what can I tell you about ...",CASES_WAV,"[-352.7203369140625, 140.18695068359375, -56.8..."
...,...,...,...,...,...,...,...,...,...,...,...
179,Video 1_6 (good).mp3,44100,2,0.76,CONTROLS,0,good,Lunch,Video 1_6 (good).wav,CONTROLS_WAV,"[-318.52154541015625, 123.62574768066406, -24...."
180,Video 1_4 (and can I have some more material).mp3,44100,2,3.60,CONTROLS,0,and can I have some more material,And can I have some more material?,Video 1_4 (and can I have some more material).wav,CONTROLS_WAV,"[-329.09893798828125, 126.80348205566406, -10...."
181,Video 1_5 (do you have a knife).mp3,44100,2,1.70,CONTROLS,0,do you have a knife,Can you hand me your knife?,Video 1_5 (do you have a knife).wav,CONTROLS_WAV,"[-274.81976318359375, 145.4311065673828, -23.5..."
182,Video 1_7 (here_s some pizza).mp3,44100,2,2.12,CONTROLS,0,here's some pizza,Here's your pizza.,Video 1_7 (here_s some pizza).wav,CONTROLS_WAV,"[-314.316650390625, 111.04605102539062, -10.59..."


In [ ]:
# Replace 'your_dataframe.xlsx' with your desired file name
excel_file_path = '/content/drive/MyDrive/VUMC_postcleft/processed_data.xlsx'
df.to_excel(excel_file_path, index=False)


# 02.use incontext learning with GPT4

In [ ]:
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Select only the required columns
df_sample = df[['hypernasality', 'original_text', 'numerical_representation']]

# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(df_sample, test_size=0.2, stratify=df_sample['hypernasality'], random_state=42)

# Sort the train DataFrame by 'hypernasality'
train_df_sorted = train_df.sort_values(by='hypernasality')

# Shuffle the test DataFrame
test_df_shuffled = shuffle(test_df, random_state=42)

# Convert the sorted train DataFrame to a list of strings formatted like JSON
train_strings = train_df_sorted.apply(lambda row: json.dumps(row.to_dict()), axis=1).tolist()

# Convert the shuffled test DataFrame to a list of strings formatted like JSON
test_strings = test_df_shuffled.apply(lambda row: json.dumps(row.to_dict()), axis=1).tolist()

# Combine everything into the 'answer' variable
answer = "\n".join(["Training Set:"] + train_strings + ["==================="] + ["Testing Set:"] + test_strings)

# Print the test set strings
print(answer)


Training Set:
{"hypernasality": 0, "original_text": "here's some pizza", "numerical_representation": [-314.316650390625, 111.04605102539062, -10.593160629272461, 39.40992736816406, -2.308807611465454, 15.177928924560547, -21.37079620361328, 21.564002990722656, -11.369519233703613, 8.297396659851074, -8.450693130493164, 6.442537784576416, 2.0681326389312744]}
{"hypernasality": 0, "original_text": "fire", "numerical_representation": [-338.3578186035156, 138.26097106933594, -20.631546020507812, 15.903144836425781, -4.744411945343018, -2.591953992843628, -22.78563690185547, 10.893670082092285, -11.19334888458252, 3.9047601222991943, 3.4270942211151123, 10.877104759216309, -3.658997058868408]}
{"hypernasality": 0, "original_text": "they don't bite me either", "numerical_representation": [-447.5281982421875, 120.810546875, -10.896355628967285, 13.836700439453125, -1.2603355646133423, -1.8610297441482544, -13.394157409667969, 3.0804758071899414, -15.047989845275879, -4.980011463165283, -9.362

In [ ]:
train_prompt = """
You are a professional speech pathologist expert trying to identify hypernasality. The following data shows if the individual has hypernasality, original text, and the numerical representation of the audio file based on using the Python function designed for extracting Mel-Frequency Cepstral Coefficients (MFCCs) from an audio file
Here is the data:

Here are the pairs of examples that do not have hypernasality("hypernasality":0):
{"hypernasality": 0, "original_text": "here's some pizza", "numerical_representation": [-314.316650390625, 111.04605102539062, -10.593160629272461, 39.40992736816406, -2.308807611465454, 15.177928924560547, -21.37079620361328, 21.564002990722656, -11.369519233703613, 8.297396659851074, -8.450693130493164, 6.442537784576416, 2.0681326389312744]}
{"hypernasality": 0, "original_text": "fire", "numerical_representation": [-338.3578186035156, 138.26097106933594, -20.631546020507812, 15.903144836425781, -4.744411945343018, -2.591953992843628, -22.78563690185547, 10.893670082092285, -11.19334888458252, 3.9047601222991943, 3.4270942211151123, 10.877104759216309, -3.658997058868408]}
{"hypernasality": 0, "original_text": "they don't bite me either", "numerical_representation": [-447.5281982421875, 120.810546875, -10.896355628967285, 13.836700439453125, -1.2603355646133423, -1.8610297441482544, -13.394157409667969, 3.0804758071899414, -15.047989845275879, -4.980011463165283, -9.362115859985352, -4.418255805969238, 1.0209296941757202]}
{"hypernasality": 0, "original_text": "good", "numerical_representation": [-318.52154541015625, 123.62574768066406, -24.474496841430664, 36.483917236328125, -13.990005493164062, 15.844413757324219, -28.559425354003906, 18.381256103515625, -11.908973693847656, 15.872629165649414, -5.0767107009887695, 8.415108680725098, 2.1506683826446533]}
{"hypernasality": 0, "original_text": "four", "numerical_representation": [-254.29165649414062, 134.97390747070312, -16.673057556152344, 2.4636192321777344, -49.434478759765625, -24.278696060180664, 9.284297943115234, 12.236101150512695, -22.868974685668945, 8.845699310302734, -12.732392311096191, -14.119959831237793, -0.1962890774011612]}
{"hypernasality": 0, "original_text": "red", "numerical_representation": [-407.8794860839844, 164.24551391601562, -13.668115615844727, 2.454789876937866, 19.42068099975586, 11.978545188903809, -19.999839782714844, 7.334909439086914, 11.949634552001953, 5.115943431854248, -0.22796577215194702, -2.4004554748535156, -7.115049362182617]}
{"hypernasality": 0, "original_text": "where is it", "numerical_representation": [-427.5153503417969, 122.91144561767578, -30.039915084838867, 14.761621475219727, -27.640409469604492, 19.022972106933594, -25.49166488647461, 11.676393508911133, -27.196657180786133, 17.65438461303711, -12.447796821594238, -8.374286651611328, 0.32886573672294617]}
{"hypernasality": 0, "original_text": "he won't fly away", "numerical_representation": [-239.58958435058594, 148.976318359375, -42.464229583740234, 6.511017799377441, -17.148487091064453, -3.9826929569244385, -9.888961791992188, 12.38595199584961, -1.296629548072815, -11.764094352722168, 0.8249483108520508, -1.2902308702468872, -2.3348240852355957]}
{"hypernasality": 0, "original_text": "Ted had a dog with white feet", "numerical_representation": [-454.2119445800781, 41.891876220703125, -79.18563842773438, 69.19869232177734, -36.21337890625, 14.137002944946289, -27.68498420715332, -27.905582427978516, 16.72525405883789, -18.027555465698242, 6.921505928039551, -16.478181838989258, -7.666031837463379]}
{"hypernasality": 0, "original_text": "can I see'", "numerical_representation": [-412.563232421875, 102.81505584716797, -28.44793701171875, 23.119646072387695, -26.13217544555664, 1.0842455625534058, -12.687361717224121, 17.613847732543945, -25.560134887695312, 14.885208129882812, -8.136920928955078, -4.889041900634766, -0.09204412251710892]}
{"hypernasality": 0, "original_text": "they feed seeds to birds", "numerical_representation": [-356.9045104980469, 20.082639694213867, -66.99832153320312, 87.00264739990234, -40.754066467285156, 30.966915130615234, -32.706092834472656, -25.92018699645996, 16.81828498840332, -25.79627227783203, 5.9286932945251465, -22.999929428100586, -11.220064163208008]}
{"hypernasality": 0, "original_text": "I played with my toys", "numerical_representation": [-353.7104797363281, 118.52685546875, -4.814436435699463, 36.47578811645508, -10.716552734375, 19.07826042175293, -12.35608196258545, 8.060433387756348, -18.46880340576172, 8.591595649719238, -11.277496337890625, 7.2258148193359375, 1.195028305053711]}
{"hypernasality": 0, "original_text": "puppies", "numerical_representation": [-331.9861755371094, 107.40956115722656, 2.092463731765747, 35.14389419555664, -5.501103401184082, 10.365973472595215, -12.07139778137207, 11.824159622192383, -11.22111701965332, 12.055030822753906, -5.313964366912842, -1.4649370908737183, -0.7818294167518616]}
{"hypernasality": 0, "original_text": "yes I'm going to give him some coffee for me and him", "numerical_representation": [-246.7664794921875, 124.92317199707031, -1.2299622297286987, 37.59580993652344, -1.8688300848007202, 10.596946716308594, -18.82198143005371, 12.927265167236328, -16.659780502319336, 3.1978371143341064, -4.052391052246094, 6.591097831726074, -0.8105242848396301]}
{"hypernasality": 0, "original_text": "I like school", "numerical_representation": [-417.1815490722656, 76.4541015625, -67.42810821533203, 49.46432876586914, -40.02613830566406, 6.2223029136657715, -25.759403228759766, -18.8490047454834, 16.15675926208496, -24.341997146606445, 7.312172889709473, -13.807418823242188, -18.306114196777344]}
{"hypernasality": 0, "original_text": "and then he ate breakfast", "numerical_representation": [-400.4725341796875, 133.96389770507812, -8.618412017822266, 45.40330123901367, 3.0036869049072266, 14.900710105895996, -11.442190170288086, 1.2485555410385132, -9.600282669067383, 13.815988540649414, -6.92319917678833, 1.8700189590454102, 6.905149459838867]}
{"hypernasality": 0, "original_text": "cookies are good to eat", "numerical_representation": [-432.45269775390625, 56.19118118286133, -89.65165710449219, 50.6878776550293, -26.550655364990234, 50.86559295654297, -1.326411247253418, -19.58587074279785, -0.5694737434387207, -45.44129943847656, -0.5626514554023743, -11.539420127868652, -13.983906745910645]}
{"hypernasality": 0, "original_text": "and where's the corn sauce", "numerical_representation": [-298.7818603515625, 121.04449462890625, -10.861045837402344, 34.415557861328125, -9.092568397521973, 13.495702743530273, -16.462934494018555, 16.791208267211914, -15.421346664428711, 6.797018527984619, -4.53266716003418, 5.657925605773926, -1.6501773595809937]}
{"hypernasality": 0, "original_text": " I swung on the swings and did other stuff", "numerical_representation": [-401.73016357421875, 121.39108276367188, -5.017462253570557, 48.19163131713867, -10.256117820739746, 27.018831253051758, -5.6894354820251465, 14.0258150100708, -22.069355010986328, 10.735372543334961, -14.798810005187988, 6.964041233062744, -2.3813140392303467]}
{"hypernasality": 0, "original_text": "its broke, I broke it", "numerical_representation": [-354.82391357421875, 135.01394653320312, 19.800321578979492, 19.247161865234375, -10.134156227111816, -1.2988495826721191, -9.679887771606445, 10.0438814163208, -13.553048133850098, -0.28000256419181824, -4.877917289733887, -1.571633219718933, 0.7445539236068726]}
{"hypernasality": 0, "original_text": "where's the burner", "numerical_representation": [-282.337646484375, 130.16944885253906, -13.882140159606934, 19.083175659179688, -11.045557975769043, 9.40736198425293, -17.30717658996582, 8.078575134277344, -12.555564880371094, 9.848686218261719, -2.1413888931274414, 7.053557872772217, -1.3683562278747559]}
{"hypernasality": 0, "original_text": "Santa came home since the snow fell", "numerical_representation": [-393.1720886230469, 42.383148193359375, -76.31912994384766, 40.35429763793945, -38.195064544677734, 30.485578536987305, -4.33135986328125, -14.015933990478516, 8.067068099975586, -41.16312026977539, -3.7182228565216064, -10.327415466308594, -14.787056922912598]}
{"hypernasality": 0, "original_text": "that's a ladybug house, right'", "numerical_representation": [-292.6459655761719, 162.96495056152344, -9.892212867736816, 6.7965989112854, 0.32117846608161926, -3.6300411224365234, 1.5623480081558228, 0.04175090417265892, -1.5930535793304443, -0.7536975145339966, 6.610475540161133, 0.21222545206546783, 2.1105704307556152]}
{"hypernasality": 0, "original_text": "they just sit on the table and I say hi to them", "numerical_representation": [-432.50982666015625, 104.15582275390625, -20.13714599609375, 20.96887969970703, -12.447052001953125, -0.253383994102478, -15.706507682800293, 8.790852546691895, -19.491559982299805, -3.871206283569336, -7.1559600830078125, 0.7276404500007629, 2.9584922790527344]}
{"hypernasality": 0, "original_text": "Do you have a brother or a sister", "numerical_representation": [-349.97845458984375, 42.96127700805664, -88.58071899414062, 69.38758087158203, -50.44060134887695, 27.153736114501953, -33.66275405883789, -30.781661987304688, 9.020245552062988, -20.24148178100586, 10.230891227722168, -23.283069610595703, -0.5099968910217285]}
{"hypernasality": 0, "original_text": "went to the park", "numerical_representation": [-408.5249328613281, 140.09500122070312, -1.731590747833252, 34.05833053588867, -10.040807723999023, 30.551782608032227, -9.038671493530273, 18.116687774658203, -27.555391311645508, 13.848367691040039, -7.738812446594238, 9.575431823730469, -5.4697699546813965]}
{"hypernasality": 0, "original_text": "do you have a knife", "numerical_representation": [-274.81976318359375, 145.4311065673828, -23.56450843811035, 27.426986694335938, -8.655342102050781, 11.443880081176758, -28.573528289794922, 6.000040054321289, -20.144359588623047, 9.884302139282227, -3.3729913234710693, 5.99395227432251, -5.874351501464844]}
{"hypernasality": 0, "original_text": "we got him", "numerical_representation": [-282.4659118652344, 160.55670166015625, -48.280479431152344, 13.813777923583984, 4.470513343811035, -4.154836654663086, -12.123940467834473, 0.6570225358009338, -3.0307207107543945, 0.22609500586986542, 7.012516975402832, 3.213090181350708, 2.4723737239837646]}
{"hypernasality": 0, "original_text": "Santa came home since the snow fell", "numerical_representation": [-403.8623352050781, 65.8228988647461, -65.19989013671875, 53.07576370239258, -38.81840896606445, 16.925071716308594, -28.962604522705078, -20.099849700927734, 18.968446731567383, -24.910308837890625, 5.350465774536133, -13.22480583190918, -13.371580123901367]}
{"hypernasality": 0, "original_text": "say just hello for pretend", "numerical_representation": [-423.87664794921875, 117.21583557128906, -20.98858070373535, 13.628201484680176, -20.847761154174805, 4.49579381942749, -13.4557523727417, 3.9337286949157715, -12.908796310424805, 1.3036390542984009, -6.493814468383789, 2.0756349563598633, 2.5965819358825684]}
{"hypernasality": 0, "original_text": "but ants do that sometimes", "numerical_representation": [-425.67425537109375, 103.9533462524414, -30.036006927490234, 14.93738842010498, -22.484331130981445, 0.2348604053258896, -12.648797035217285, 10.80567741394043, -16.309589385986328, 3.255459785461426, -7.603976726531982, -2.946770429611206, 1.6439640522003174]}
{"hypernasality": 0, "original_text": "but", "numerical_representation": [-449.6979064941406, 127.45062255859375, -15.62159252166748, 9.72036075592041, 0.5251541137695312, -11.16162395477295, -7.030332565307617, -1.2912312746047974, -15.217567443847656, 5.811631679534912, 1.7696393728256226, 3.248781204223633, 1.1840388774871826]}
{"hypernasality": 0, "original_text": "Playing in the snow is fun", "numerical_representation": [-434.7479553222656, 40.0706901550293, -63.758602142333984, 53.500083923339844, -38.87538528442383, 21.64339256286621, -27.039508819580078, -28.667055130004883, 14.37502670288086, -16.064762115478516, 7.188129425048828, -16.7347469329834, -11.438759803771973]}
{"hypernasality": 0, "original_text": "the polar bears are dancing", "numerical_representation": [-375.46209716796875, 132.05885314941406, -30.23935317993164, 45.11177062988281, -3.8405349254608154, 17.458755493164062, -8.90272045135498, 5.6317009925842285, -19.9200439453125, 13.251362800598145, -12.186144828796387, 2.3031222820281982, -3.6116600036621094]}
{"hypernasality": 0, "original_text": "thank you ray", "numerical_representation": [-367.9347229003906, 140.21067810058594, 21.508378982543945, 34.178138732910156, 17.151376724243164, -0.11750185489654541, -2.3793206214904785, 5.833749294281006, -1.2407126426696777, -0.6940250396728516, -1.2693442106246948, 2.086831569671631, 1.8973941802978516]}
{"hypernasality": 0, "original_text": "stop", "numerical_representation": [-313.61553955078125, 120.6831283569336, -25.875019073486328, 18.01205062866211, -22.018268585205078, 10.940808296203613, -8.366928100585938, 20.127222061157227, -19.387556076049805, 4.240433692932129, -2.9882919788360596, 2.4313950538635254, -1.4407602548599243]}
{"hypernasality": 0, "original_text": "fill up the gas", "numerical_representation": [-250.58831787109375, 119.58746337890625, -15.703017234802246, 22.694578170776367, -20.6768741607666, 6.448459625244141, -18.997150421142578, 11.727448463439941, -17.133892059326172, 2.9780428409576416, -7.395652770996094, -0.11304324120283127, 3.4894261360168457]}
{"hypernasality": 0, "original_text": "sue roasted a duck for supper", "numerical_representation": [-438.1555480957031, 68.48530578613281, -63.778282165527344, 46.03855514526367, -38.455543518066406, 19.815645217895508, -8.496828079223633, -25.548078536987305, 3.1368863582611084, -29.163156509399414, 5.350287914276123, -12.221370697021484, -16.210838317871094]}
{"hypernasality": 0, "original_text": "can I have your phone", "numerical_representation": [-413.536376953125, 118.32636260986328, -29.235788345336914, 20.50141716003418, -28.890586853027344, 12.430008888244629, -29.024248123168945, 21.572669982910156, -22.138620376586914, 7.603678226470947, -10.025592803955078, 5.154943466186523, 1.7061090469360352]}
{"hypernasality": 0, "original_text": "its all of our birthdays", "numerical_representation": [-347.2049865722656, 136.45054626464844, -5.801585674285889, 28.247520446777344, -15.445128440856934, 7.5924482345581055, -20.47747039794922, 18.041349411010742, -14.566655158996582, 9.642721176147461, 0.9859180450439453, 5.3191118240356445, -4.678378105163574]}
{"hypernasality": 0, "original_text": "green", "numerical_representation": [-446.6971130371094, 151.7830047607422, 7.997592449188232, 34.498268127441406, 16.804105758666992, 12.794358253479004, -15.0126371383667, 13.481039047241211, -0.985371470451355, -4.758467674255371, -10.405121803283691, -12.434035301208496, -7.994369983673096]}
{"hypernasality": 0, "original_text": "shh", "numerical_representation": [-365.45556640625, 83.18830108642578, -58.1092529296875, 96.62307739257812, -18.87989044189453, 25.172157287597656, -26.686193466186523, 41.697147369384766, -14.908170700073242, 21.327648162841797, -10.770071983337402, 17.065122604370117, -1.6829782724380493]}
{"hypernasality": 0, "original_text": "like drums", "numerical_representation": [-474.50836181640625, 130.09548950195312, -25.078960418701172, 24.560285568237305, -34.33063888549805, 22.878835678100586, -23.339405059814453, 15.188733100891113, -17.694276809692383, 5.936470985412598, -9.9996337890625, 3.559056282043457, -2.6307594776153564]}
{"hypernasality": 0, "original_text": "can I play with Jack", "numerical_representation": [-341.03216552734375, 124.30364227294922, -18.103300094604492, 24.429088592529297, -7.226626396179199, 12.365540504455566, -14.96137809753418, 15.637368202209473, -11.782940864562988, 16.596982955932617, 4.831630229949951, 5.291296482086182, -1.1635993719100952]}
{"hypernasality": 0, "original_text": "yes I am going to give him coffee for me and him", "numerical_representation": [-260.47991943359375, 126.72785949707031, -1.91536283493042, 38.849971771240234, -2.5923781394958496, 11.907004356384277, -19.703704833984375, 14.18496322631836, -17.445951461791992, 4.036054611206055, -4.667425155639648, 6.349365711212158, -1.2512357234954834]}
{"hypernasality": 0, "original_text": "bugs and spiders, I protect", "numerical_representation": [-447.767822265625, 120.7002182006836, -16.513750076293945, 13.677332878112793, -11.202004432678223, 3.693255662918091, -7.758529186248779, 8.99756145477295, -11.656825065612793, 2.2279467582702637, -4.500323295593262, -1.91822350025177, 2.7268731594085693]}
{"hypernasality": 0, "original_text": "pretend it stops running when the car is going", "numerical_representation": [-285.3266906738281, 130.88201904296875, -13.24666690826416, 22.49609375, -5.9247822761535645, 3.147162437438965, -23.445343017578125, 7.383640766143799, -16.10611915588379, 3.7399742603302, -2.9784889221191406, 1.0270171165466309, -1.8998304605484009]}
{"hypernasality": 0, "original_text": "puppy are you ready'", "numerical_representation": [-285.2287902832031, 122.35124206542969, 1.6412885189056396, 34.39849090576172, 2.661123514175415, 9.258344650268555, -16.405778884887695, 12.386581420898438, -17.14431381225586, 9.882874488830566, -1.8811094760894775, 2.768132448196411, -5.864969253540039]}
{"hypernasality": 0, "original_text": "it stops going", "numerical_representation": [-291.5823059082031, 118.32067108154297, 7.273540019989014, 41.81880187988281, -15.642449378967285, 11.602429389953613, -26.49786376953125, 13.665165901184082, -17.639511108398438, 5.431158542633057, -6.372157096862793, 6.261345386505127, 3.0369811058044434]}
{"hypernasality": 0, "original_text": "shh", "numerical_representation": [-300.9380798339844, 65.16253662109375, -48.775272369384766, 88.26013946533203, -13.762866973876953, 38.30063247680664, -35.489234924316406, 26.9569091796875, -5.558383941650391, 22.711292266845703, -11.845114707946777, 13.600346565246582, -0.11734190583229065]}
{"hypernasality": 0, "original_text": "pizza bundt", "numerical_representation": [-322.443115234375, 120.5947494506836, 0.1731337457895279, 43.03276443481445, -13.833467483520508, 15.370514869689941, -17.02610206604004, 13.2139892578125, -17.10870361328125, 10.76021671295166, -5.005504131317139, 8.54093074798584, 0.0032221477013081312]}
{"hypernasality": 0, "original_text": "We shouldn't play in the street", "numerical_representation": [-427.39129638671875, 22.005971908569336, -73.8521957397461, 80.3757553100586, -28.907922744750977, 22.301300048828125, -33.0462532043457, -32.6959114074707, 15.682881355285645, -19.821327209472656, 3.8845620155334473, -20.830951690673828, -9.180371284484863]}
{"hypernasality": 0, "original_text": "I said thank you ray", "numerical_representation": [-357.11773681640625, 157.96510314941406, 21.506696701049805, 36.728572845458984, -9.621773719787598, 0.4053577184677124, -2.644402027130127, -1.4745879173278809, 3.8685317039489746, 8.09870433807373, -0.5470772981643677, -6.933988571166992, 3.615912675857544]}
{"hypernasality": 0, "original_text": "no they dont talk", "numerical_representation": [-401.18603515625, 116.6793212890625, -22.272993087768555, -0.16545504331588745, -18.124414443969727, -4.6717681884765625, -8.826055526733398, 17.452545166015625, -15.869908332824707, -3.1495256423950195, -5.28016471862793, -9.061944007873535, -1.9677380323410034]}
{"hypernasality": 0, "original_text": "and then he brushed his teeth", "numerical_representation": [-397.6322937011719, 132.51934814453125, -13.837478637695312, 49.86563491821289, 1.155307412147522, 17.326908111572266, -13.94123649597168, 3.693964719772339, -8.240595817565918, 13.68974781036377, -8.61807632446289, 1.0149117708206177, 7.287909984588623]}
{"hypernasality": 0, "original_text": "three times", "numerical_representation": [-448.9305725097656, 124.9089126586914, -28.820083618164062, 20.86905288696289, -27.314910888671875, 20.13602638244629, -22.705141067504883, 15.147699356079102, -21.100242614746094, 14.105178833007812, -12.670620918273926, -3.158113956451416, -1.784797191619873]}
{"hypernasality": 0, "original_text": "and can I have some more material", "numerical_representation": [-329.09893798828125, 126.80348205566406, -10.270654678344727, 32.802825927734375, -2.5149192810058594, 8.153069496154785, -17.59292221069336, 6.914050579071045, -18.339109420776367, 4.585212707519531, -5.299656867980957, 5.890632629394531, 0.6953542828559875]}
{"hypernasality": 0, "original_text": "and then go to school", "numerical_representation": [-436.5776062011719, 141.796630859375, 8.682721138000488, 37.777435302734375, -2.9015135765075684, 10.882061004638672, -6.353466987609863, 1.7952097654342651, -15.099238395690918, 8.212226867675781, -4.6318182945251465, -1.0954734086990356, 6.254204273223877]}
{"hypernasality": 0, "original_text": "look at this book with us", "numerical_representation": [-339.9624938964844, 51.35060501098633, -68.62452697753906, 46.7181282043457, -42.5457878112793, 25.428028106689453, -11.937192916870117, -4.865809917449951, 6.060321807861328, -43.259979248046875, 0.24141812324523926, -17.193164825439453, -14.7529296875]}
{"hypernasality": 0, "original_text": "she wore blue shoes", "numerical_representation": [-459.67205810546875, 64.83921813964844, -78.25238037109375, 47.41449737548828, -32.858360290527344, 39.47542190551758, 2.5287740230560303, -14.1272611618042, 4.100606918334961, -40.92561721801758, -7.4191999435424805, -14.025945663452148, -11.829744338989258]}
{"hypernasality": 0, "original_text": "and then he was a boy", "numerical_representation": [-406.9534912109375, 146.90074157714844, 3.9690308570861816, 36.478271484375, 7.146398067474365, 7.955857276916504, -8.713133811950684, -4.649252891540527, -16.708974838256836, 6.1102166175842285, -9.453404426574707, -5.098039627075195, 5.933281898498535]}
{"hypernasality": 0, "original_text": "where's my bowl", "numerical_representation": [-352.494873046875, 115.89765930175781, 8.758959770202637, 31.86680793762207, -3.2998571395874023, 16.6189022064209, -13.925167083740234, 19.147565841674805, -9.88599967956543, 10.693219184875488, -4.122896671295166, 7.783102035522461, 0.3602350950241089]}
{"hypernasality": 0, "original_text": "I like school", "numerical_representation": [-446.3446044921875, 67.27718353271484, -58.79469680786133, 31.829740524291992, -36.48221969604492, 27.87257957458496, -8.52711296081543, -10.696024894714355, 4.480653762817383, -38.14439010620117, -0.037501368671655655, -11.945509910583496, -12.252113342285156]}
{"hypernasality": 0, "original_text": "blue", "numerical_representation": [-466.1670227050781, 149.3057861328125, 25.58269500732422, 29.690635681152344, 11.956026077270508, 17.43624496459961, 0.10555347800254822, 11.199718475341797, -12.366458892822266, -3.4355785846710205, -1.0144522190093994, -7.913032054901123, -5.159607410430908]}
{"hypernasality": 0, "original_text": "cookies are good to eat", "numerical_representation": [-424.9427795410156, 70.25720977783203, -80.35193634033203, 66.79078674316406, -13.61523723602295, 18.97321128845215, -24.068161010742188, -26.324806213378906, 4.889316558837891, -30.37301254272461, 5.108522415161133, -17.174131393432617, -15.907066345214844]}
{"hypernasality": 0, "original_text": "play some games", "numerical_representation": [-469.5064392089844, 128.10183715820312, -28.63483238220215, 33.01948547363281, -33.65666961669922, 17.5338077545166, -26.427867889404297, 21.116029739379883, -23.758695602416992, 5.430513858795166, -11.385053634643555, 3.6845483779907227, -3.2284657955169678]}
{"hypernasality": 0, "original_text": "yellow", "numerical_representation": [-411.3094787597656, 172.1162567138672, -12.7103910446167, 22.0598087310791, -8.377355575561523, -6.320786476135254, 0.8331826329231262, 16.96478271484375, -14.999632835388184, -3.421651601791382, -3.884734630584717, -0.9957895278930664, 1.9380824565887451]}
{"hypernasality": 0, "original_text": "who, who", "numerical_representation": [-367.56854248046875, 113.39887237548828, 16.60821533203125, 35.99468994140625, -2.233116388320923, 23.4719181060791, -11.095778465270996, 19.774526596069336, -11.37217903137207, 12.410862922668457, -4.811723232269287, 7.606529712677002, 1.6612811088562012]}
{"hypernasality": 0, "original_text": "but if its not, then we would stop running", "numerical_representation": [-321.0458984375, 133.01263427734375, -7.294881343841553, 25.517040252685547, 1.7289882898330688, 6.954270839691162, -21.162858963012695, 9.078413963317871, -16.078493118286133, 3.8178329467773438, -0.27331116795539856, 5.441389083862305, -3.4825265407562256]}
{"hypernasality": 0, "original_text": "Most Boys Like to Play Football", "numerical_representation": [-318.7271423339844, 37.15936279296875, -90.73167419433594, 75.86328125, -56.01700210571289, 22.146650314331055, -26.643962860107422, -25.849468231201172, 13.826654434204102, -24.00981330871582, 9.344793319702148, -16.601652145385742, -4.382903575897217]}

Here are the pairs of examples that have hypernasality
{"hypernasality": 1, "original_text": "do you have a brother or a sister", "numerical_representation": [-365.8101806640625, 112.98062896728516, -80.7423324584961, 54.39430236816406, -37.18933868408203, 24.43941879272461, -30.19179916381836, -26.587697982788086, 12.764906883239746, -36.93735122680664, -3.2487778663635254, -5.3106608390808105, -3.388093948364258]}
{"hypernasality": 1, "original_text": "we shouldn't play in the street", "numerical_representation": [-358.0149230957031, 98.59503936767578, -102.51810455322266, 45.563629150390625, -7.632992744445801, 24.10328483581543, -28.30856704711914, -20.273757934570312, 7.925145626068115, -29.963287353515625, -3.915360450744629, -23.4755802154541, -5.047581195831299]}
{"hypernasality": 1, "original_text": "buy baby a bib", "numerical_representation": [-416.8902282714844, 65.76830291748047, -48.922725677490234, 64.48535919189453, -22.664466857910156, 35.64484786987305, -30.31867218017578, -27.246328353881836, 6.3100504875183105, -31.4154109954834, 10.16001033782959, -14.023831367492676, -6.279337406158447]}
{"hypernasality": 1, "original_text": "do you have a brother or a sister", "numerical_representation": [-346.6312561035156, 74.79408264160156, -50.96628189086914, 76.03170013427734, -57.67744827270508, 20.562105178833008, -22.732789993286133, -29.678672790527344, 8.978156089782715, -24.931034088134766, 7.644595146179199, -12.062199592590332, -7.259523868560791]}
{"hypernasality": 1, "original_text": "go get the cake", "numerical_representation": [-438.13140869140625, 75.57269287109375, -93.7221450805664, 65.74007415771484, -50.8034553527832, 10.4592866897583, 3.6142077445983887, -33.66188049316406, 9.288922309875488, -19.579296112060547, 7.238318920135498, 3.3185932636260986, -4.710987091064453]}
{"hypernasality": 1, "original_text": "buy bobby a puppy", "numerical_representation": [-375.5430603027344, 149.4890899658203, -24.179845809936523, 17.298702239990234, -6.092220306396484, 8.14392375946045, -28.46038818359375, 0.9680064916610718, -26.959012985229492, -12.51950454711914, -15.65468978881836, -14.448335647583008, -2.75262713432312]}
{"hypernasality": 1, "original_text": "puppy", "numerical_representation": [-415.67010498046875, 130.9578857421875, -8.50123405456543, 25.675012588500977, -6.014280319213867, 4.851944923400879, -24.42441749572754, -1.3791236877441406, -23.42071533203125, -6.316161632537842, -13.720210075378418, -9.680319786071777, -1.7845888137817383]}
{"hypernasality": 1, "original_text": "susie sees the scissors", "numerical_representation": [-395.76031494140625, 59.5333366394043, -87.97545623779297, 67.60498809814453, -61.65242004394531, 10.033586502075195, 6.152714252471924, -35.40304183959961, 8.75303840637207, -14.620068550109863, 14.469558715820312, -2.798903703689575, -16.824092864990234]}
{"hypernasality": 1, "original_text": "bobby", "numerical_representation": [-420.1449890136719, 126.79886627197266, -8.975176811218262, 28.724077224731445, -3.6780805587768555, 6.434601306915283, -22.588041305541992, 10.2804594039917, -19.880390167236328, -7.57263708114624, -12.978315353393555, 4.069015026092529, -11.619115829467773]}
{"hypernasality": 1, "original_text": "put the baby in the buggy", "numerical_representation": [-301.32366943359375, 167.06239318847656, -48.06004333496094, 39.40061950683594, 16.764263153076172, -28.707605361938477, 4.804245471954346, -8.331219673156738, -16.100276947021484, -17.41008758544922, -14.262218475341797, -0.47529855370521545, -18.027414321899414]}
{"hypernasality": 1, "original_text": "don't wear your dirty", "numerical_representation": [-312.03387451171875, 132.21551513671875, 8.819872856140137, 37.434051513671875, 12.767495155334473, 10.701306343078613, -11.264548301696777, -0.16676531732082367, -11.022507667541504, -13.583122253417969, 5.602464199066162, -11.239553451538086, -10.177659034729004]}
{"hypernasality": 1, "original_text": "take teddy to town", "numerical_representation": [-284.4936828613281, 177.8997344970703, -50.79314422607422, 41.86826705932617, 11.465666770935059, -32.146854400634766, 4.551002502441406, -12.72150707244873, -13.220337867736816, -15.444199562072754, -19.56302833557129, -1.6257613897323608, -16.76551628112793]}
{"hypernasality": 1, "original_text": "we go swimming on a very hot day", "numerical_representation": [-360.3680114746094, 135.0343017578125, -82.39039611816406, 29.16556167602539, -9.550189018249512, 9.413025856018066, -18.611003875732422, -25.69336700439453, -2.2641208171844482, -26.66054344177246, 4.921998977661133, -8.009827613830566, -12.866047859191895]}
{"hypernasality": 1, "original_text": "most boys like to play football", "numerical_representation": [-375.3310546875, 124.65213012695312, -72.97403717041016, 30.2695255279541, -19.14194107055664, 3.6294162273406982, -20.020023345947266, -23.248273849487305, 1.45475172996521, -24.928543090820312, 8.912957191467285, -6.481081485748291, -16.04928970336914]}
{"hypernasality": 1, "original_text": "and I started out on the switch board and two phones", "numerical_representation": [-364.57672119140625, 104.95849609375, -91.53890228271484, 41.045074462890625, -28.527536392211914, 1.6022074222564697, -6.5273966789245605, -23.14741325378418, 15.048556327819824, -20.75473976135254, -5.936681747436523, -11.550904273986816, -19.94339370727539]}
{"hypernasality": 1, "original_text": "pamper", "numerical_representation": [-383.6915283203125, 149.5983123779297, -4.011986255645752, 11.069225311279297, -29.116989135742188, 1.4555612802505493, -24.759981155395508, 4.470621109008789, -22.959068298339844, -7.248496055603027, -12.418022155761719, -18.154523849487305, 0.328230082988739]}
{"hypernasality": 1, "original_text": "take a teddybear", "numerical_representation": [-387.82281494140625, 50.84046936035156, -86.16191101074219, 60.563411712646484, -57.90447235107422, 8.462337493896484, -1.673393726348877, -43.36799240112305, 10.070115089416504, -11.946380615234375, 5.359221935272217, 9.495433807373047, -5.5987396240234375]}
{"hypernasality": 1, "original_text": "playing in the snow is fun", "numerical_representation": [-341.087890625, 102.67699432373047, -90.27838134765625, 33.45213317871094, -21.560882568359375, 22.02403450012207, -9.169219970703125, -20.920503616333008, 3.4536187648773193, -32.32588577270508, -1.270689606666565, -17.228412628173828, -11.726662635803223]}
{"hypernasality": 1, "original_text": "most boys like to play football", "numerical_representation": [-363.5120849609375, 103.98828125, -67.05109405517578, 64.01362609863281, -46.23469924926758, 14.489520072937012, -37.35353088378906, -20.003129959106445, 14.92878246307373, -36.60646057128906, -4.869020462036133, -9.599348068237305, -2.270085096359253]}
{"hypernasality": 1, "original_text": "I struggle with my 's'", "numerical_representation": [-299.2655944824219, 139.1504669189453, -35.9262580871582, 33.54214096069336, -18.150697708129883, 19.107540130615234, -5.3570098876953125, 10.340896606445312, -3.3921871185302734, 8.127460479736328, -6.199929714202881, -3.069051742553711, -5.355368137359619]}
{"hypernasality": 1, "original_text": "sissy sees the sun", "numerical_representation": [-339.2411804199219, 131.85240173339844, -41.59820556640625, 32.20408630371094, 4.767225742340088, 4.281380653381348, -36.13886642456055, -11.469762802124023, -20.260284423828125, -15.897211074829102, -12.298218727111816, -18.58487319946289, -3.671539306640625]}
{"hypernasality": 1, "original_text": "ca, ca, ca", "numerical_representation": [-387.8785400390625, 176.17625427246094, -79.96920013427734, -41.78838348388672, -17.785167694091797, -21.56556510925293, 3.066035509109497, -13.85888385772705, -26.90378189086914, -18.120529174804688, -8.763895988464355, -1.6018388271331787, -10.41575813293457]}
{"hypernasality": 1, "original_text": "take a turtle", "numerical_representation": [-396.6440734863281, 78.46405029296875, -89.21305847167969, 44.72099685668945, -58.7353401184082, 16.74384117126465, 18.235614776611328, -48.02248001098633, -1.6119863986968994, -3.1222081184387207, 3.8020849227905273, 5.624581813812256, 1.652413249015808]}
{"hypernasality": 1, "original_text": "told", "numerical_representation": [-372.43096923828125, 134.2797088623047, 3.7779757976531982, 39.13146209716797, -0.21132458746433258, 21.373485565185547, -4.4631733894348145, 9.733072280883789, -6.2123799324035645, 9.030061721801758, 2.8687679767608643, -0.9676824808120728, -0.22720806300640106]}
{"hypernasality": 1, "original_text": "but it will help me with the 's'", "numerical_representation": [-360.2122802734375, 133.91384887695312, -20.429046630859375, 18.683025360107422, -19.493927001953125, 9.015814781188965, -17.519577026367188, 11.965103149414062, -6.477949142456055, -5.023278713226318, 0.8314717411994934, 2.4758410453796387, -11.91892147064209]}
{"hypernasality": 1, "original_text": "take a tire", "numerical_representation": [-407.1781921386719, 82.2300796508789, -98.05654907226562, 54.31171798706055, -52.989444732666016, 21.87215805053711, 2.6737098693847656, -47.38117599487305, 9.367864608764648, -6.951462268829346, 8.988415718078613, 6.79561710357666, -1.4891421794891357]}
{"hypernasality": 1, "original_text": "go get the car", "numerical_representation": [-409.41851806640625, 62.0489387512207, -96.1219253540039, 48.132572174072266, -54.570621490478516, 21.34701156616211, 20.80027198791504, -33.49856185913086, 1.8749579191207886, -15.440200805664062, 11.318939208984375, 6.983205318450928, -4.780111312866211]}
{"hypernasality": 1, "original_text": "ted had a dog with white feet", "numerical_representation": [-374.8000793457031, 68.70259857177734, -48.61258316040039, 79.90442657470703, -44.60945510864258, 23.327159881591797, -29.163816452026367, -32.82921600341797, 11.830832481384277, -25.10882568359375, 2.6133852005004883, -13.203009605407715, -7.349518299102783]}
{"hypernasality": 1, "original_text": "daddy", "numerical_representation": [-293.267822265625, 175.19532775878906, -63.047576904296875, 44.347572326660156, 12.651880264282227, -32.982215881347656, 6.306475639343262, -5.72115421295166, -3.7090697288513184, -11.076757431030273, -16.06711769104004, -3.1617116928100586, -22.692365646362305]}
{"hypernasality": 1, "original_text": "Nick's grandmother lives in the city", "numerical_representation": [-372.54571533203125, 105.98299407958984, -100.81196594238281, 36.41577911376953, -13.348716735839844, 24.091594696044922, -16.736597061157227, -23.618253707885742, 2.766108751296997, -26.43929100036621, -1.4964203834533691, -20.642963409423828, -10.51402473449707]}
{"hypernasality": 1, "original_text": "we went to town yesterday", "numerical_representation": [-333.219970703125, 76.42379760742188, -89.12235260009766, 72.77510070800781, -33.55392074584961, 34.317665100097656, -41.9411506652832, -24.621646881103516, 12.909443855285645, -26.140270233154297, 17.41616439819336, -13.753273010253906, 1.5732824802398682]}
{"hypernasality": 1, "original_text": "Nick's grandmother lives in the city", "numerical_representation": [-341.5704040527344, 117.7804183959961, -82.73017120361328, 39.3297233581543, -7.169986724853516, 7.9362664222717285, -18.06960678100586, -26.698486328125, -0.248784139752388, -25.23788833618164, 4.426325798034668, -8.221550941467285, -12.700133323669434]}
{"hypernasality": 1, "original_text": "she went shopping", "numerical_representation": [-320.1985168457031, 173.38185119628906, -50.82009506225586, 38.97981262207031, 9.080086708068848, -32.29557800292969, 4.017061710357666, -10.420249938964844, -10.582344055175781, -14.872701644897461, -16.45984649658203, 1.232380747795105, -16.991186141967773]}
{"hypernasality": 1, "original_text": "in the safe deposit department", "numerical_representation": [-367.6443176269531, 88.59137725830078, -98.41595458984375, 54.22686004638672, -35.18558120727539, 23.367847442626953, -9.940560340881348, -34.73014831542969, 3.1682052612304688, -22.731233596801758, 12.389275550842285, -6.769416809082031, -18.770797729492188]}
{"hypernasality": 1, "original_text": "I like ice cream", "numerical_representation": [-360.55035400390625, 113.56474304199219, -124.6507568359375, 38.888614654541016, -21.007783889770508, 28.93172836303711, -13.920366287231445, -23.016002655029297, 8.04061222076416, -30.87677574157715, -3.156888484954834, -22.808576583862305, -4.014507293701172]}
{"hypernasality": 1, "original_text": "transferring, getting information to customers", "numerical_representation": [-389.47833251953125, 90.45024871826172, -82.5062026977539, 51.50310516357422, -19.862924575805664, 6.046064376831055, -7.180060386657715, -32.41193389892578, 9.536974906921387, -21.829181671142578, -5.409058094024658, -7.943378925323486, -23.765636444091797]}
{"hypernasality": 1, "original_text": "we shouldn't play in the street", "numerical_representation": [-386.1136779785156, 71.1166763305664, -77.1468505859375, 77.74676513671875, -31.12228775024414, 45.467628479003906, -40.20235061645508, -32.15999221801758, 10.26469612121582, -24.516809463500977, 15.320591926574707, -14.139183044433594, 1.710428237915039]}
{"hypernasality": 1, "original_text": "whatever they need", "numerical_representation": [-429.7159729003906, 104.78227996826172, -63.175071716308594, 45.14907455444336, -9.582267761230469, 10.628982543945312, -19.560680389404297, -23.75464630126953, 13.266484260559082, -27.032608032226562, -6.385513782501221, -17.054302215576172, -21.418882369995117]}
{"hypernasality": 1, "original_text": "puffy", "numerical_representation": [-392.8885803222656, 127.11654663085938, -17.941465377807617, 34.87961959838867, 3.3761751651763916, 7.0897297859191895, -28.435779571533203, -1.8051549196243286, -21.922544479370117, -12.55938720703125, -14.928671836853027, -13.982091903686523, -2.753791093826294]}
{"hypernasality": 1, "original_text": "playing in the snow is fun", "numerical_representation": [-356.08221435546875, 78.30680847167969, -58.911705017089844, 87.2978286743164, -49.86753463745117, 17.183813095092773, -30.754968643188477, -31.2528076171875, 5.190479278564453, -29.170072555541992, 6.062982559204102, -14.309144020080566, -12.027122497558594]}
{"hypernasality": 1, "original_text": "see the seesaw", "numerical_representation": [-420.5852966308594, 143.90725708007812, -42.68655776977539, -25.76106834411621, -25.983684539794922, -39.69658279418945, -6.696603298187256, 4.160006046295166, -9.632880210876465, -27.839616775512695, -22.526508331298828, 0.7876307368278503, -4.422415256500244]}
{"hypernasality": 1, "original_text": "opening and closing boxes", "numerical_representation": [-399.4700927734375, 100.7889404296875, -89.0400619506836, 62.50117874145508, -37.45604705810547, 5.466788291931152, -0.0758233517408371, -29.066308975219727, 17.33189582824707, -19.032855987548828, -9.59627914428711, -11.985814094543457, -17.20016860961914]}
{"hypernasality": 1, "original_text": "Tom had ham and eggs for breakfast", "numerical_representation": [-335.3016052246094, 114.93402862548828, -93.89409637451172, 40.99630355834961, -9.668865203857422, 10.72458267211914, -29.88217544555664, -28.907184600830078, 3.2081289291381836, -25.271472930908203, 10.661558151245117, -6.47737455368042, -13.23298168182373]}
{"hypernasality": 1, "original_text": "muddy", "numerical_representation": [-360.6280822753906, 131.1361541748047, 16.786144256591797, 38.10710906982422, 13.29775333404541, 9.284567832946777, 6.275053024291992, 7.534638404846191, -0.8849759697914124, -8.87618350982666, 0.022029124200344086, -2.5568788051605225, -10.129690170288086]}
{"hypernasality": 1, "original_text": "pick up the baby", "numerical_representation": [-417.6043395996094, 58.079261779785156, -76.64094543457031, 56.04108428955078, -44.914512634277344, 17.034465789794922, 3.653635263442993, -44.76927185058594, 7.3062663078308105, -9.28344440460205, -0.18823854625225067, 2.4732983112335205, -4.05325174331665]}
{"hypernasality": 1, "original_text": "muddy", "numerical_representation": [-360.44549560546875, 122.8425521850586, 9.935541152954102, 40.345794677734375, 8.788330078125, 20.087156295776367, -7.227380275726318, 11.951920509338379, -7.85575008392334, -7.6207709312438965, 2.7472341060638428, -3.8675460815429688, -2.9933199882507324]}
{"hypernasality": 1, "original_text": "buy baby a bib", "numerical_representation": [-404.6099548339844, 124.63481903076172, -10.003445625305176, 26.237165451049805, 0.6652342677116394, 4.89296293258667, -22.86432647705078, 1.36211097240448, -16.31204605102539, -5.9409565925598145, -8.396892547607422, 4.428510665893555, -8.330262184143066]}
{"hypernasality": 1, "original_text": "Tom had ham and eggs for breakfast", "numerical_representation": [-330.2029724121094, 101.66614532470703, -112.32727813720703, 29.8204288482666, -16.214168548583984, 25.144620895385742, -16.54768943786621, -19.891841888427734, 6.734830856323242, -29.8120059967041, -0.7535023093223572, -20.33193588256836, -10.346235275268555]}
{"hypernasality": 1, "original_text": "pick up the pie", "numerical_representation": [-406.44427490234375, 80.14720153808594, -101.77937316894531, 51.99897766113281, -58.53972244262695, 21.895977020263672, 10.891718864440918, -52.5419807434082, 10.556069374084473, 2.7238686084747314, 9.328439712524414, -1.2791478633880615, -6.876070976257324]}
{"hypernasality": 1, "original_text": "pa, pa, pa", "numerical_representation": [-563.2019653320312, 142.91305541992188, -16.097564697265625, -35.53030014038086, 0.6750016808509827, 0.6655504107475281, -13.904097557067871, -19.573768615722656, -28.991924285888672, -34.30912780761719, -17.200504302978516, 0.3602227568626404, 2.5906007289886475]}
{"hypernasality": 1, "original_text": "sha, sha, sha", "numerical_representation": [-409.0993957519531, 171.3542938232422, -72.66511535644531, -36.452117919921875, -18.24212646484375, -15.983219146728516, -3.173325300216675, -13.640625, -2.47741961479187, -18.18174171447754, -19.515676498413086, 3.728663206100464, -15.077213287353516]}
{"hypernasality": 1, "original_text": "ted and todd", "numerical_representation": [-316.9971008300781, 130.7137908935547, -6.498289585113525, 38.7363395690918, 7.07015323638916, 14.30289363861084, -16.556499481201172, 13.383818626403809, -9.263466835021973, 1.4366819858551025, 1.0402942895889282, -4.293432712554932, -1.7576160430908203]}
{"hypernasality": 1, "original_text": "bah", "numerical_representation": [-432.41650390625, 106.71757507324219, -10.667421340942383, 24.18528938293457, 0.7825651168823242, 10.245043754577637, -12.756349563598633, 11.171695709228516, -13.060823440551758, -4.9485087394714355, -10.509066581726074, 1.2741566896438599, -8.208060264587402]}
{"hypernasality": 1, "original_text": "susie sees the dress", "numerical_representation": [-389.94683837890625, 72.77706146240234, -98.228515625, 58.92106628417969, -59.35301208496094, 17.86792755126953, 16.650056838989258, -35.194183349609375, 2.1525399684906006, -19.213533401489258, 9.960487365722656, -6.116108417510986, -16.47696304321289]}
{"hypernasality": 1, "original_text": "One, Two, Three, Four, Five, Six, Seven, Eight, Nine", "numerical_representation": [-394.56683349609375, 154.70797729492188, -5.230838298797607, -5.922043800354004, 1.0436718463897705, -7.049663066864014, -4.62546443939209, -0.4752768278121948, 4.014153480529785, -4.381707668304443, -4.090364933013916, 2.1081714630126953, -1.4631404876708984]}
{"hypernasality": 1, "original_text": "bah, bah, bah", "numerical_representation": [-559.2308959960938, 129.19631958007812, 7.932275295257568, -22.601755142211914, -5.055710315704346, -3.2604146003723145, -9.664192199707031, -14.078676223754883, -25.80918312072754, -34.811222076416016, -22.11506462097168, -0.5055290460586548, 6.993997097015381]}
{"hypernasality": 1, "original_text": "most boys like to play football", "numerical_representation": [-336.84033203125, 76.53764343261719, -77.54268646240234, 72.84976959228516, -50.24864196777344, 33.466922760009766, -31.877628326416016, -19.64858627319336, 10.32419204711914, -26.222898483276367, 11.213007926940918, -18.591136932373047, 6.458547592163086]}
{"hypernasality": 1, "original_text": "playing in the snow is fun", "numerical_representation": [-384.7137451171875, 95.62788391113281, -62.04146194458008, 58.2069206237793, -34.092838287353516, 18.875886917114258, -28.14879608154297, -26.10211181640625, 8.432037353515625, -34.029823303222656, -2.4359705448150635, -4.564362049102783, -4.325794219970703]}
{"hypernasality": 1, "original_text": "my other son is a sophomore in high school", "numerical_representation": [-404.3597106933594, 135.65782165527344, -23.537734985351562, 39.63490676879883, -41.46494674682617, -3.5665030479431152, -39.46125793457031, -38.5875129699707, 14.13664436340332, -14.348220825195312, 2.0272958278656006, -8.550093650817871, -8.849629402160645]}
{"hypernasality": 1, "original_text": "do it for daddy", "numerical_representation": [-295.1346740722656, 164.9134979248047, -57.728675842285156, 38.152767181396484, 15.237326622009277, -28.10862159729004, 4.491347312927246, -11.459328651428223, -12.04990005493164, -13.993938446044922, -17.017757415771484, -2.150902032852173, -16.737680435180664]}
{"hypernasality": 1, "original_text": "smashing", "numerical_representation": [-379.40069580078125, 105.71591186523438, -27.943758010864258, 35.28929138183594, 1.9608664512634277, 24.409732818603516, 0.8331077098846436, 19.203628540039062, -1.3716788291931152, 8.068726539611816, 2.550123453140259, 5.38639497756958, 0.6561414003372192]}
{"hypernasality": 1, "original_text": "playing in the snow is fun", "numerical_representation": [-375.6810302734375, 66.8976821899414, -73.3096923828125, 63.705657958984375, -34.02982711791992, 39.850460052490234, -26.219772338867188, -30.833812713623047, 4.347980976104736, -22.999752044677734, 22.448650360107422, -10.326220512390137, -8.283700942993164]}
{"hypernasality": 1, "original_text": " Three, Four, Five, Six, Seven, Eight, Nine", "numerical_representation": [-562.5535278320312, 81.67758178710938, 0.6105219721794128, -0.2865557372570038, 6.995032787322998, -15.572976112365723, -17.454410552978516, -5.504492282867432, -12.52878475189209, -19.883108139038086, -13.312808990478516, -4.0246148109436035, 3.597581624984741]}
{"hypernasality": 1, "original_text": "feet", "numerical_representation": [-345.8603515625, 119.72769927978516, -19.695598602294922, 21.39253044128418, -1.4547771215438843, 9.25846004486084, -6.454497814178467, 8.668696403503418, -1.4259713888168335, 9.978684425354004, -3.644078493118286, 0.4594776928424835, 0.5038045644760132]}
{"hypernasality": 1, "original_text": "well it will help me", "numerical_representation": [-347.89544677734375, 149.8165283203125, -32.30320358276367, 15.939460754394531, -16.079269409179688, -1.8741638660430908, -10.375349044799805, 8.039054870605469, -6.294971466064453, -4.095771789550781, -1.504103422164917, -2.6138358116149902, -6.378047466278076]}
{"hypernasality": 1, "original_text": "pick up the books", "numerical_representation": [-343.1074523925781, 42.84730911254883, -112.54253387451172, 62.82022476196289, -52.75916290283203, 43.88491439819336, 14.647490501403809, -62.36105728149414, 7.092282295227051, 0.6643553376197815, 2.7123870849609375, -3.3221921920776367, 4.21776008605957]}
{"hypernasality": 1, "original_text": "go get the cookie", "numerical_representation": [-446.4930725097656, 56.093875885009766, -75.55620574951172, 48.34809875488281, -54.65376281738281, 15.439019203186035, 17.5849666595459, -48.55686950683594, 11.78730297088623, 7.809458255767822, 4.352871417999268, -9.357528686523438, -19.48804473876953]}
{"hypernasality": 1, "original_text": "mom", "numerical_representation": [-362.99615478515625, 119.8902587890625, 1.7868438959121704, 21.94952392578125, -2.12851881980896, 2.488628625869751, -23.29805564880371, 4.587810039520264, -28.703351974487305, -17.11809730529785, -8.142827987670898, 7.799583911895752, -10.234053611755371]}
{"hypernasality": 1, "original_text": "Popeye", "numerical_representation": [-386.7400207519531, 129.2578582763672, -8.776902198791504, 16.136987686157227, 6.69942569732666, 2.5906436443328857, -16.216007232666016, 1.2319377660751343, -22.67780876159668, -16.816621780395508, -8.538122177124023, 5.424983024597168, -5.5298848152160645]}
{"hypernasality": 1, "original_text": "susie sees the horse", "numerical_representation": [-421.3475341796875, 83.75125122070312, -91.84088134765625, 60.73478317260742, -58.69849395751953, 10.999898910522461, 4.327905178070068, -33.84489440917969, 8.036823272705078, -13.897279739379883, 20.423608779907227, -2.8941497802734375, -20.74188232421875]}
{"hypernasality": 1, "original_text": "baby", "numerical_representation": [-269.9215393066406, 188.19383239746094, -63.25454330444336, 55.54095458984375, 13.21756362915039, -48.30703353881836, -0.4753384590148926, -5.021027565002441, -24.675037384033203, -23.999664306640625, -11.605193138122559, 2.505388021469116, -25.433500289916992]}
{"hypernasality": 1, "original_text": "one is in school in Arizona, his father is going out there this weekend to check on him", "numerical_representation": [-382.554443359375, 135.40472412109375, -38.100624084472656, 26.173768997192383, -29.291597366333008, 9.839670181274414, -32.026668548583984, -38.672183990478516, 7.787450313568115, -18.840309143066406, -0.41306209564208984, -9.657365798950195, -9.281317710876465]}
{"hypernasality": 1, "original_text": "puppy will pull a rope", "numerical_representation": [-408.851806640625, 122.74169158935547, 7.652419567108154, 26.7182559967041, -2.1805620193481445, 1.1952553987503052, -19.869518280029297, 4.301959991455078, -19.85444450378418, -8.472549438476562, -12.588227272033691, -0.8361210227012634, -5.087459087371826]}
{"hypernasality": 1, "original_text": "tie and hat to eat dinner", "numerical_representation": [-326.7718200683594, 129.97842407226562, -9.34816837310791, 35.22370529174805, 16.26540756225586, 9.898303031921387, -11.240264892578125, 7.167028903961182, -4.903318881988525, -5.796239852905273, 3.69940447807312, -1.830039143562317, -7.771651744842529]}
{"hypernasality": 1, "original_text": "ted had a dog with white feet", "numerical_representation": [-369.8966369628906, 96.01170349121094, -67.9906234741211, 57.426692962646484, -34.754520416259766, 17.108522415161133, -27.576433181762695, -24.717369079589844, 8.094891548156738, -35.26839065551758, -0.014075695537030697, -3.031226873397827, -8.442068099975586]}
{"hypernasality": 1, "original_text": "I ride a choo-choo train", "numerical_representation": [-330.19482421875, 174.1708526611328, -52.113590240478516, 39.4134521484375, 9.297197341918945, -28.359785079956055, 7.2279744148254395, -12.721929550170898, -13.316128730773926, -15.670340538024902, -15.319265365600586, 1.0097116231918335, -18.733610153198242]}
{"hypernasality": 1, "original_text": "seesaw", "numerical_representation": [-500.0505065917969, 174.15478515625, -34.518489837646484, -38.01487731933594, -35.119224548339844, -24.556621551513672, -0.42623355984687805, -1.9588346481323242, -12.549692153930664, -30.68667221069336, -18.179594039916992, 2.1632988452911377, -6.067529201507568]}
"""

In [ ]:
test_prompt = """
Based on the learned examples, please tell me if these examples have hypernasality by filling in 0 for no hypernasality and 1 for hypernasality:

{"hypernasality": '', "original_text": "ma, ma, ma", "numerical_representation": [-523.4454345703125, 160.59765625, 16.638282775878906, -30.011714935302734, -10.421371459960938, -2.767976999282837, -8.125978469848633, -14.461625099182129, -28.49098777770996, -37.47283172607422, -24.131996154785156, -0.033182017505168915, 10.096097946166992]}
{"hypernasality": '', "original_text": "go get the wagon", "numerical_representation": [-296.84967041015625, 173.03860473632812, -60.14041519165039, 32.46588134765625, 8.220316886901855, -33.22257614135742, 3.5051662921905518, -10.685630798339844, -10.770039558410645, -12.330078125, -13.874552726745605, 1.3827120065689087, -16.66341781616211]}
{"hypernasality": '', "original_text": "bye-bye", "numerical_representation": [-423.54351806640625, 125.05402374267578, -53.93264389038086, 14.763923645019531, -23.252044677734375, -0.8204927444458008, -11.162549018859863, 13.89601993560791, -25.459745407104492, 11.402987480163574, -11.471709251403809, -2.916095018386841, 7.420944690704346]}
{"hypernasality": '', "original_text": "you have pants on", "numerical_representation": [-410.84613037109375, 133.83084106445312, -48.67580795288086, 3.214205026626587, -17.03330421447754, 15.247991561889648, -19.38698959350586, 18.059171676635742, -20.802377700805664, 10.588240623474121, -12.88408088684082, -3.92319917678833, -7.307775974273682]}
{"hypernasality": '', "original_text": "we dont have any pineapple", "numerical_representation": [-330.7674865722656, 126.60137176513672, -1.7072526216506958, 30.21820831298828, -1.420722484588623, 23.688396453857422, -17.51378631591797, 17.018861770629883, -12.657621383666992, 8.099841117858887, -6.446732997894287, 10.039698600769043, -2.2834982872009277]}
{"hypernasality": '', "original_text": "they feed seeds to birds", "numerical_representation": [-382.4083251953125, 33.9415283203125, -81.6314468383789, 59.446868896484375, -28.36338996887207, 46.76637268066406, -3.0324559211730957, -13.22170639038086, 1.1267650127410889, -50.72395706176758, -5.058108806610107, -12.587112426757812, -14.213844299316406]}
{"hypernasality": '', "original_text": "where'd dad go", "numerical_representation": [-356.1424255371094, 189.09002685546875, -6.606091499328613, -2.824500799179077, 13.51265811920166, 5.362226486206055, -4.00547456741333, 4.706381320953369, -13.06798267364502, 17.65468978881836, 4.6345014572143555, -13.154485702514648, 0.33895665407180786]}
{"hypernasality": '', "original_text": "we have to have it first, then the cake", "numerical_representation": [-342.3438415527344, 115.67051696777344, -14.946680068969727, 38.13743591308594, 1.6433895826339722, 13.097597122192383, -20.159610748291016, 17.834095001220703, -6.485004425048828, 10.746752738952637, -2.499163866043091, 3.518080711364746, 1.610191822052002]}
{"hypernasality": '', "original_text": "yeah", "numerical_representation": [-303.99395751953125, 157.67599487304688, -32.78515625, 7.214668273925781, 0.3239806890487671, 3.563629627227783, -2.655731678009033, 0.5036352276802063, 2.212688684463501, -2.6115643978118896, 5.481392860412598, 4.7245073318481445, 4.63879919052124]}
{"hypernasality": '', "original_text": "we shouldn't play in the street", "numerical_representation": [-360.77130126953125, 69.30449676513672, -72.25696563720703, 93.55999755859375, -48.24067306518555, 30.813613891601562, -27.517166137695312, -32.52854537963867, 11.149147033691406, -19.76921844482422, 8.048664093017578, -18.145227432250977, -9.127974510192871]}
{"hypernasality": '', "original_text": "don't open it, its locked", "numerical_representation": [-321.10186767578125, 183.04229736328125, -28.90302848815918, 14.896389961242676, -4.827393531799316, 1.6416829824447632, -1.9311575889587402, 2.2149806022644043, 0.29115039110183716, -1.8978137969970703, 8.705763816833496, 0.8945053815841675, 1.8515903949737549]}
{"hypernasality": '', "original_text": "papi are you ready", "numerical_representation": [-254.4236602783203, 128.3041229248047, -2.4457175731658936, 34.223392486572266, 1.6233785152435303, 7.7787370681762695, -19.276323318481445, 12.630143165588379, -18.679189682006836, 9.987550735473633, -2.5068295001983643, 2.0418901443481445, -6.524878025054932]}
{"hypernasality": '', "original_text": "the pizza", "numerical_representation": [-314.0162048339844, 125.21251678466797, 12.543079376220703, 50.6954460144043, -8.691390991210938, 13.00108814239502, -24.69085121154785, 13.379531860351562, -17.395397186279297, 5.464900016784668, -6.699265956878662, 3.054283857345581, -6.8947343826293945]}
{"hypernasality": '', "original_text": "2 more in grammar school, and out little 3 year old", "numerical_representation": [-370.3482666015625, 138.3352813720703, -31.247291564941406, 34.08516311645508, -37.12031936645508, 9.474739074707031, -32.537864685058594, -39.81637954711914, 6.1931867599487305, -21.524757385253906, 0.24033717811107635, -10.142782211303711, -11.365910530090332]}
{"hypernasality": '', "original_text": "I want a picture of giant man", "numerical_representation": [-406.03558349609375, 155.23277282714844, -23.409860610961914, 6.408907890319824, 2.211758613586426, 22.8829402923584, 0.8069822192192078, 9.334912300109863, -2.1408519744873047, 10.860602378845215, 4.628368377685547, 8.630574226379395, 2.643878221511841]}
{"hypernasality": '', "original_text": "shiny shoes", "numerical_representation": [-428.20208740234375, 170.6973114013672, -26.201948165893555, -14.629733085632324, -1.7046778202056885, -21.013221740722656, -5.242834568023682, 7.289400100708008, 0.6467728614807129, -17.717361450195312, -14.461783409118652, 6.840051174163818, -0.2739943861961365]}
{"hypernasality": '', "original_text": "door and four", "numerical_representation": [-382.3828430175781, 126.8658218383789, 2.6599481105804443, 22.07579803466797, -10.917302131652832, -4.049671649932861, -7.4766459465026855, 12.919812202453613, -11.1893949508667, 3.804414987564087, -6.449995994567871, 1.8796497583389282, -2.9324982166290283]}
{"hypernasality": '', "original_text": "bobby", "numerical_representation": [-383.9786682128906, 134.16455078125, -10.061858177185059, 32.857337951660156, -1.7600347995758057, 1.8291510343551636, -33.92999267578125, 3.835007905960083, -20.973037719726562, -12.673851013183594, -15.08671760559082, 5.832170009613037, -7.978484630584717]}
{"hypernasality": '', "original_text": "we shouldn't play in the street", "numerical_representation": [-399.544189453125, 99.68357849121094, -65.57657623291016, 64.10311126708984, -30.399555206298828, 21.508577346801758, -27.20697021484375, -22.62975311279297, 7.149099349975586, -33.20420455932617, 2.406451463699341, -5.968044281005859, -5.575110912322998]}
{"hypernasality": '', "original_text": "Jack likes cheese sandwiches for lunch", "numerical_representation": [-364.8030700683594, 78.18758392333984, -81.21894073486328, 67.09992980957031, -35.39420700073242, 35.69902038574219, -38.92142868041992, -29.731779098510742, 9.233311653137207, -31.417312622070312, 16.626758575439453, -10.279197692871094, -2.894139051437378]}
{"hypernasality": '', "original_text": "shoe", "numerical_representation": [-423.3542785644531, 184.0464630126953, -28.8523006439209, -15.903599739074707, -7.103864669799805, -21.7088623046875, -0.5761725306510925, 6.461463928222656, -1.9085222482681274, -17.71940803527832, -16.94540786743164, -2.6188244819641113, -7.845320224761963]}
{"hypernasality": '', "original_text": "she wore blue shoes", "numerical_representation": [-424.87591552734375, 61.505210876464844, -74.57488250732422, 68.97763061523438, -23.42822265625, 19.187156677246094, -17.82511329650879, -17.19542121887207, 9.305635452270508, -34.5945930480957, 2.45770263671875, -15.363945960998535, -17.38882827758789]}
{"hypernasality": '', "original_text": "I eat cake and ice cream", "numerical_representation": [-300.2364501953125, 167.2384033203125, -53.73587417602539, 34.38330078125, 13.942227363586426, -29.147171020507812, 1.3223940134048462, -11.072997093200684, -11.872092247009277, -15.191774368286133, -15.719659805297852, 0.8768016695976257, -16.137418746948242]}
{"hypernasality": '', "original_text": "sixty, sixty, sixty", "numerical_representation": [-430.0491638183594, 71.0531234741211, -78.1672592163086, 65.36420440673828, -48.989219665527344, 1.250731110572815, -3.410470485687256, -28.20243263244629, 15.031984329223633, -21.089475631713867, 11.865164756774902, 5.758316993713379, -19.220863342285156]}
{"hypernasality": '', "original_text": "oh let me see, what can I tell you about them now", "numerical_representation": [-352.7203369140625, 140.18695068359375, -56.83113098144531, 29.861135482788086, -43.04221725463867, 14.096572875976562, -43.3836555480957, -33.55165100097656, 12.864432334899902, -21.50568199157715, 2.047971725463867, -12.755514144897461, -5.814598560333252]}
{"hypernasality": '', "original_text": "Ted had a dog with white feet", "numerical_representation": [-355.2947692871094, 126.66532897949219, -81.57817840576172, 46.66024398803711, -12.314688682556152, 9.016596794128418, -25.211322784423828, -29.202939987182617, 0.9677002429962158, -25.100873947143555, 7.078424453735352, -10.444621086120605, -11.307692527770996]}
{"hypernasality": '', "original_text": "tires", "numerical_representation": [-276.2408447265625, 112.18889617919922, -23.925968170166016, 30.01728630065918, -13.210087776184082, 2.497563123703003, -27.281970977783203, 5.231518745422363, -23.487932205200195, 6.888440132141113, -1.1979820728302002, 0.8535549640655518, -3.2456305027008057]}
{"hypernasality": '', "original_text": "look at this book with us", "numerical_representation": [-293.05322265625, 43.28155517578125, -71.25052642822266, 67.0547103881836, -59.32616424560547, 21.169645309448242, -26.999614715576172, -19.60069465637207, 15.46639347076416, -27.597280502319336, 4.576290607452393, -17.16154670715332, -3.033902645111084]}
{"hypernasality": '', "original_text": "fence", "numerical_representation": [-306.1473693847656, 160.445068359375, -15.51785945892334, 14.945609092712402, -10.23627758026123, 9.057631492614746, -10.152568817138672, 11.501471519470215, -7.855470180511475, 4.2354278564453125, -3.6323137283325195, -0.11877725273370743, -7.913267612457275]}
{"hypernasality": '', "original_text": "most boys like to play football", "numerical_representation": [-315.1574401855469, 49.205055236816406, -64.50914001464844, 97.6524429321289, -61.47548294067383, 25.347225189208984, -38.05367660522461, -34.87712097167969, 12.913159370422363, -24.749351501464844, 15.605209350585938, -7.901693820953369, -9.200847625732422]}
{"hypernasality": '', "original_text": "And I'll put some", "numerical_representation": [-311.5871887207031, 116.30655670166016, -13.06558609008789, 51.28089904785156, -18.925148010253906, 29.258419036865234, -26.245088577270508, 17.33948516845703, -18.896528244018555, 11.546334266662598, -7.05389928817749, 6.46613073348999, -1.1219942569732666]}
{"hypernasality": '', "original_text": "Most boys like to play football", "numerical_representation": [-374.01104736328125, 114.7081298828125, -105.17838287353516, 29.335086822509766, -13.795332908630371, 14.178194999694824, -8.70284652709961, -18.79938507080078, 0.5424554944038391, -29.456989288330078, -4.423678874969482, -24.319074630737305, -9.252711296081543]}
{"hypernasality": '', "original_text": "where's the sauce'", "numerical_representation": [-314.5812072753906, 107.02288818359375, 0.4859614372253418, 37.96367645263672, -9.298739433288574, 9.864797592163086, -11.697738647460938, 16.330055236816406, -9.170608520507812, 7.633543968200684, -2.2035162448883057, 1.7346971035003662, 0.5361695289611816]}
{"hypernasality": '', "original_text": "I see the sun in the sky", "numerical_representation": [-298.8079833984375, 167.41249084472656, -51.63760757446289, 32.506858825683594, 14.491250038146973, -28.896282196044922, -0.7446603178977966, -12.195006370544434, -10.354391098022461, -12.784646987915039, -17.36318588256836, 0.4322144687175751, -13.09603214263916]}
{"hypernasality": '', "original_text": "do you like milk in your tea'", "numerical_representation": [-302.3052978515625, 125.90079498291016, 10.325353622436523, 40.403228759765625, -0.3169914186000824, 4.696054935455322, -24.120771408081055, 3.4888932704925537, -16.151493072509766, 5.659123420715332, -6.488191604614258, 1.2609331607818604, 3.960608959197998]}
{"hypernasality": '', "original_text": "I say hello to spiders because they're nice to me", "numerical_representation": [-422.32073974609375, 102.55937957763672, -15.3735933303833, 19.50240135192871, -12.644205093383789, -3.461423635482788, -8.580442428588867, 0.24333436787128448, -13.523791313171387, -2.7428009510040283, -10.832101821899414, -3.4438271522521973, 4.2313761711120605]}
{"hypernasality": '', "original_text": "it was really fun, I think it really helps me, it makes me feel great", "numerical_representation": [-345.7295837402344, 121.9375, -18.8929386138916, 22.960472106933594, -7.592164039611816, 21.083581924438477, -9.688018798828125, 11.517437934875488, -9.584684371948242, 5.571232795715332, -2.6793711185455322, -2.1362898349761963, 1.2854808568954468]}
"""

In [ ]:
import openai

# Set your API key here
openai.api_key = 'api'

def incontext_slp():
    # Get the definition for the specified question type
    client = openai.OpenAI(api_key='api')


    # Call the OpenAI API using the updated method
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        max_tokens= 4000,
        messages=[
            {"role": "assistant", "content": train_prompt},
            {"role": "user", "content": test_prompt}
        ]
    )

    # Extract and return the response
    #output = response['choices'][0]['message']['content']
    output = response.choices[0].message.content
    return output

output = incontext_slp()
output

'Based on the patterns observed in the provided examples, here are the predicted labels (0 for no hypernasality, 1 for hypernasality) for each instance:\n\n1. {"hypernasality": 1, "original_text": "ma, ma, ma"}\n2. {"hypernasality": 1, "original_text": "go get the wagon"}\n3. {"hypernasality": 1, "original_text": "bye-bye"}\n4. {"hypernasality": 1, "original_text": "you have pants on"}\n5. {"hypernasality": 0, "original_text": "we dont have any pineapple"}\n6. {"hypernasality": 0, "original_text": "they feed seeds to birds"}\n7. {"hypernasality": 1, "original_text": "where\'d dad go"}\n8. {"hypernasality": 0, "original_text": "we have to have it first, then the cake"}\n9. {"hypernasality": 0, "original_text": "yeah"}\n10. {"hypernasality": 1, "original_text": "we shouldn\'t play in the street"}\n11. {"hypernasality": 0, "original_text": "don\'t open it, its locked"}\n12. {"hypernasality": 0, "original_text": "papi are you ready"}\n13. {"hypernasality": 0, "original_text": "the pizza"}\

In [ ]:
print(output)

Based on the patterns observed in the provided examples, here are the predicted labels (0 for no hypernasality, 1 for hypernasality) for each instance:

1. {"hypernasality": 1, "original_text": "ma, ma, ma"}
2. {"hypernasality": 1, "original_text": "go get the wagon"}
3. {"hypernasality": 1, "original_text": "bye-bye"}
4. {"hypernasality": 1, "original_text": "you have pants on"}
5. {"hypernasality": 0, "original_text": "we dont have any pineapple"}
6. {"hypernasality": 0, "original_text": "they feed seeds to birds"}
7. {"hypernasality": 1, "original_text": "where'd dad go"}
8. {"hypernasality": 0, "original_text": "we have to have it first, then the cake"}
9. {"hypernasality": 0, "original_text": "yeah"}
10. {"hypernasality": 1, "original_text": "we shouldn't play in the street"}
11. {"hypernasality": 0, "original_text": "don't open it, its locked"}
12. {"hypernasality": 0, "original_text": "papi are you ready"}
13. {"hypernasality": 0, "original_text": "the pizza"}
14. {"hypernasalit

In [ ]:
string1 = """
{"hypernasality": 1, "original_text": "ma, ma, ma", "numerical_representation": [-523.4454345703125, 160.59765625, 16.638282775878906, -30.011714935302734, -10.421371459960938, -2.767976999282837, -8.125978469848633, -14.461625099182129, -28.49098777770996, -37.47283172607422, -24.131996154785156, -0.033182017505168915, 10.096097946166992]}
{"hypernasality": 1, "original_text": "go get the wagon", "numerical_representation": [-296.84967041015625, 173.03860473632812, -60.14041519165039, 32.46588134765625, 8.220316886901855, -33.22257614135742, 3.5051662921905518, -10.685630798339844, -10.770039558410645, -12.330078125, -13.874552726745605, 1.3827120065689087, -16.66341781616211]}
{"hypernasality": 0, "original_text": "bye-bye", "numerical_representation": [-423.54351806640625, 125.05402374267578, -53.93264389038086, 14.763923645019531, -23.252044677734375, -0.8204927444458008, -11.162549018859863, 13.89601993560791, -25.459745407104492, 11.402987480163574, -11.471709251403809, -2.916095018386841, 7.420944690704346]}
{"hypernasality": 0, "original_text": "you have pants on", "numerical_representation": [-410.84613037109375, 133.83084106445312, -48.67580795288086, 3.214205026626587, -17.03330421447754, 15.247991561889648, -19.38698959350586, 18.059171676635742, -20.802377700805664, 10.588240623474121, -12.88408088684082, -3.92319917678833, -7.307775974273682]}
{"hypernasality": 0, "original_text": "we dont have any pineapple", "numerical_representation": [-330.7674865722656, 126.60137176513672, -1.7072526216506958, 30.21820831298828, -1.420722484588623, 23.688396453857422, -17.51378631591797, 17.018861770629883, -12.657621383666992, 8.099841117858887, -6.446732997894287, 10.039698600769043, -2.2834982872009277]}
{"hypernasality": 0, "original_text": "they feed seeds to birds", "numerical_representation": [-382.4083251953125, 33.9415283203125, -81.6314468383789, 59.446868896484375, -28.36338996887207, 46.76637268066406, -3.0324559211730957, -13.22170639038086, 1.1267650127410889, -50.72395706176758, -5.058108806610107, -12.587112426757812, -14.213844299316406]}
{"hypernasality": 0, "original_text": "where'd dad go", "numerical_representation": [-356.1424255371094, 189.09002685546875, -6.606091499328613, -2.824500799179077, 13.51265811920166, 5.362226486206055, -4.00547456741333, 4.706381320953369, -13.06798267364502, 17.65468978881836, 4.6345014572143555, -13.154485702514648, 0.33895665407180786]}
{"hypernasality": 0, "original_text": "we have to have it first, then the cake", "numerical_representation": [-342.3438415527344, 115.67051696777344, -14.946680068969727, 38.13743591308594, 1.6433895826339722, 13.097597122192383, -20.159610748291016, 17.834095001220703, -6.485004425048828, 10.746752738952637, -2.499163866043091, 3.518080711364746, 1.610191822052002]}
{"hypernasality": 0, "original_text": "yeah", "numerical_representation": [-303.99395751953125, 157.67599487304688, -32.78515625, 7.214668273925781, 0.3239806890487671, 3.563629627227783, -2.655731678009033, 0.5036352276802063, 2.212688684463501, -2.6115643978118896, 5.481392860412598, 4.7245073318481445, 4.63879919052124]}
{"hypernasality": 1, "original_text": "we shouldn't play in the street", "numerical_representation": [-360.77130126953125, 69.30449676513672, -72.25696563720703, 93.55999755859375, -48.24067306518555, 30.813613891601562, -27.517166137695312, -32.52854537963867, 11.149147033691406, -19.76921844482422, 8.048664093017578, -18.145227432250977, -9.127974510192871]}
{"hypernasality": 0, "original_text": "don't open it, its locked", "numerical_representation": [-321.10186767578125, 183.04229736328125, -28.90302848815918, 14.896389961242676, -4.827393531799316, 1.6416829824447632, -1.9311575889587402, 2.2149806022644043, 0.29115039110183716, -1.8978137969970703, 8.705763816833496, 0.8945053815841675, 1.8515903949737549]}
{"hypernasality": 0, "original_text": "papi are you ready", "numerical_representation": [-254.4236602783203, 128.3041229248047, -2.4457175731658936, 34.223392486572266, 1.6233785152435303, 7.7787370681762695, -19.276323318481445, 12.630143165588379, -18.679189682006836, 9.987550735473633, -2.5068295001983643, 2.0418901443481445, -6.524878025054932]}
{"hypernasality": 0, "original_text": "the pizza", "numerical_representation": [-314.0162048339844, 125.21251678466797, 12.543079376220703, 50.6954460144043, -8.691390991210938, 13.00108814239502, -24.69085121154785, 13.379531860351562, -17.395397186279297, 5.464900016784668, -6.699265956878662, 3.054283857345581, -6.8947343826293945]}
{"hypernasality": 1, "original_text": "2 more in grammar school, and out little 3 year old", "numerical_representation": [-370.3482666015625, 138.3352813720703, -31.247291564941406, 34.08516311645508, -37.12031936645508, 9.474739074707031, -32.537864685058594, -39.81637954711914, 6.1931867599487305, -21.524757385253906, 0.24033717811107635, -10.142782211303711, -11.365910530090332]}
{"hypernasality": 1, "original_text": "I want a picture of giant man", "numerical_representation": [-406.03558349609375, 155.23277282714844, -23.409860610961914, 6.408907890319824, 2.211758613586426, 22.8829402923584, 0.8069822192192078, 9.334912300109863, -2.1408519744873047, 10.860602378845215, 4.628368377685547, 8.630574226379395, 2.643878221511841]}
{"hypernasality": 1, "original_text": "shiny shoes", "numerical_representation": [-428.20208740234375, 170.6973114013672, -26.201948165893555, -14.629733085632324, -1.7046778202056885, -21.013221740722656, -5.242834568023682, 7.289400100708008, 0.6467728614807129, -17.717361450195312, -14.461783409118652, 6.840051174163818, -0.2739943861961365]}
{"hypernasality": 0, "original_text": "door and four", "numerical_representation": [-382.3828430175781, 126.8658218383789, 2.6599481105804443, 22.07579803466797, -10.917302131652832, -4.049671649932861, -7.4766459465026855, 12.919812202453613, -11.1893949508667, 3.804414987564087, -6.449995994567871, 1.8796497583389282, -2.9324982166290283]}
{"hypernasality": 1, "original_text": "bobby", "numerical_representation": [-383.9786682128906, 134.16455078125, -10.061858177185059, 32.857337951660156, -1.7600347995758057, 1.8291510343551636, -33.92999267578125, 3.835007905960083, -20.973037719726562, -12.673851013183594, -15.08671760559082, 5.832170009613037, -7.978484630584717]}
{"hypernasality": 1, "original_text": "we shouldn't play in the street", "numerical_representation": [-399.544189453125, 99.68357849121094, -65.57657623291016, 64.10311126708984, -30.399555206298828, 21.508577346801758, -27.20697021484375, -22.62975311279297, 7.149099349975586, -33.20420455932617, 2.406451463699341, -5.968044281005859, -5.575110912322998]}
{"hypernasality": 1, "original_text": "Jack likes cheese sandwiches for lunch", "numerical_representation": [-364.8030700683594, 78.18758392333984, -81.21894073486328, 67.09992980957031, -35.39420700073242, 35.69902038574219, -38.92142868041992, -29.731779098510742, 9.233311653137207, -31.417312622070312, 16.626758575439453, -10.279197692871094, -2.894139051437378]}
{"hypernasality": 1, "original_text": "shoe", "numerical_representation": [-423.3542785644531, 184.0464630126953, -28.8523006439209, -15.903599739074707, -7.103864669799805, -21.7088623046875, -0.5761725306510925, 6.461463928222656, -1.9085222482681274, -17.71940803527832, -16.94540786743164, -2.6188244819641113, -7.845320224761963]}
{"hypernasality": 0, "original_text": "she wore blue shoes", "numerical_representation": [-424.87591552734375, 61.505210876464844, -74.57488250732422, 68.97763061523438, -23.42822265625, 19.187156677246094, -17.82511329650879, -17.19542121887207, 9.305635452270508, -34.5945930480957, 2.45770263671875, -15.363945960998535, -17.38882827758789]}
{"hypernasality": 1, "original_text": "I eat cake and ice cream", "numerical_representation": [-300.2364501953125, 167.2384033203125, -53.73587417602539, 34.38330078125, 13.942227363586426, -29.147171020507812, 1.3223940134048462, -11.072997093200684, -11.872092247009277, -15.191774368286133, -15.719659805297852, 0.8768016695976257, -16.137418746948242]}
{"hypernasality": 1, "original_text": "sixty, sixty, sixty", "numerical_representation": [-430.0491638183594, 71.0531234741211, -78.1672592163086, 65.36420440673828, -48.989219665527344, 1.250731110572815, -3.410470485687256, -28.20243263244629, 15.031984329223633, -21.089475631713867, 11.865164756774902, 5.758316993713379, -19.220863342285156]}
{"hypernasality": 1, "original_text": "oh let me see, what can I tell you about them now", "numerical_representation": [-352.7203369140625, 140.18695068359375, -56.83113098144531, 29.861135482788086, -43.04221725463867, 14.096572875976562, -43.3836555480957, -33.55165100097656, 12.864432334899902, -21.50568199157715, 2.047971725463867, -12.755514144897461, -5.814598560333252]}
{"hypernasality": 1, "original_text": "Ted had a dog with white feet", "numerical_representation": [-355.2947692871094, 126.66532897949219, -81.57817840576172, 46.66024398803711, -12.314688682556152, 9.016596794128418, -25.211322784423828, -29.202939987182617, 0.9677002429962158, -25.100873947143555, 7.078424453735352, -10.444621086120605, -11.307692527770996]}
{"hypernasality": 0, "original_text": "tires", "numerical_representation": [-276.2408447265625, 112.18889617919922, -23.925968170166016, 30.01728630065918, -13.210087776184082, 2.497563123703003, -27.281970977783203, 5.231518745422363, -23.487932205200195, 6.888440132141113, -1.1979820728302002, 0.8535549640655518, -3.2456305027008057]}
{"hypernasality": 0, "original_text": "look at this book with us", "numerical_representation": [-293.05322265625, 43.28155517578125, -71.25052642822266, 67.0547103881836, -59.32616424560547, 21.169645309448242, -26.999614715576172, -19.60069465637207, 15.46639347076416, -27.597280502319336, 4.576290607452393, -17.16154670715332, -3.033902645111084]}
{"hypernasality": 1, "original_text": "fence", "numerical_representation": [-306.1473693847656, 160.445068359375, -15.51785945892334, 14.945609092712402, -10.23627758026123, 9.057631492614746, -10.152568817138672, 11.501471519470215, -7.855470180511475, 4.2354278564453125, -3.6323137283325195, -0.11877725273370743, -7.913267612457275]}
{"hypernasality": 1, "original_text": "most boys like to play football", "numerical_representation": [-315.1574401855469, 49.205055236816406, -64.50914001464844, 97.6524429321289, -61.47548294067383, 25.347225189208984, -38.05367660522461, -34.87712097167969, 12.913159370422363, -24.749351501464844, 15.605209350585938, -7.901693820953369, -9.200847625732422]}
{"hypernasality": 0, "original_text": "And I'll put some", "numerical_representation": [-311.5871887207031, 116.30655670166016, -13.06558609008789, 51.28089904785156, -18.925148010253906, 29.258419036865234, -26.245088577270508, 17.33948516845703, -18.896528244018555, 11.546334266662598, -7.05389928817749, 6.46613073348999, -1.1219942569732666]}
{"hypernasality": 1, "original_text": "Most boys like to play football", "numerical_representation": [-374.01104736328125, 114.7081298828125, -105.17838287353516, 29.335086822509766, -13.795332908630371, 14.178194999694824, -8.70284652709961, -18.79938507080078, 0.5424554944038391, -29.456989288330078, -4.423678874969482, -24.319074630737305, -9.252711296081543]}
{"hypernasality": 0, "original_text": "where's the sauce'", "numerical_representation": [-314.5812072753906, 107.02288818359375, 0.4859614372253418, 37.96367645263672, -9.298739433288574, 9.864797592163086, -11.697738647460938, 16.330055236816406, -9.170608520507812, 7.633543968200684, -2.2035162448883057, 1.7346971035003662, 0.5361695289611816]}
{"hypernasality": 1, "original_text": "I see the sun in the sky", "numerical_representation": [-298.8079833984375, 167.41249084472656, -51.63760757446289, 32.506858825683594, 14.491250038146973, -28.896282196044922, -0.7446603178977966, -12.195006370544434, -10.354391098022461, -12.784646987915039, -17.36318588256836, 0.4322144687175751, -13.09603214263916]}
{"hypernasality": 0, "original_text": "do you like milk in your tea'", "numerical_representation": [-302.3052978515625, 125.90079498291016, 10.325353622436523, 40.403228759765625, -0.3169914186000824, 4.696054935455322, -24.120771408081055, 3.4888932704925537, -16.151493072509766, 5.659123420715332, -6.488191604614258, 1.2609331607818604, 3.960608959197998]}
{"hypernasality": 0, "original_text": "I say hello to spiders because they're nice to me", "numerical_representation": [-422.32073974609375, 102.55937957763672, -15.3735933303833, 19.50240135192871, -12.644205093383789, -3.461423635482788, -8.580442428588867, 0.24333436787128448, -13.523791313171387, -2.7428009510040283, -10.832101821899414, -3.4438271522521973, 4.2313761711120605]}
{"hypernasality": 1, "original_text": "it was really fun, I think it really helps me, it makes me feel great", "numerical_representation": [-345.7295837402344, 121.9375, -18.8929386138916, 22.960472106933594, -7.592164039611816, 21.083581924438477, -9.688018798828125, 11.517437934875488, -9.584684371948242, 5.571232795715332, -2.6793711185455322, -2.1362898349761963, 1.2854808568954468]}

"""

## Accuracy of 59.46%